## IMPORTACIÓ INFORMACIÓ DE SABIO-RK

Generem un document csv on hi guardem els camps SabioReaction ID, Name, Role i Smiles de totes les reaccions químiques presents a la base de dades.

In [157]:
#install
import requests
import pandas as pd 

from urllib.request import urlopen
from urllib.parse import quote

def CIRconvert(ids):
    try:
        url = 'http://cactus.nci.nih.gov/chemical/structure/' + quote(ids) + '/smiles'
        ans = urlopen(url).read().decode('utf8')
        return ans
    except:
        return 'Did not work'


import os.path 

datafile = "data/smiles.csv"

if os.path.exists(datafile): 
    print(datafile, " exists") 
else: 
    print("Generating ",datafile)

    QUERY_URL = 'http://sabiork.h-its.org/sabioRestWebServices/searchReactionParticipants'

    query = {"SabioReactionID":"*", "fields[]":["Name","Role", "Smiles"]}

    request = requests.post(QUERY_URL, params = query) 
    request.raise_for_status() 

    print(request.text)

    # results

    fh=open("data/smiles.csv", "w")

    fh.write(request.text)







data/smiles.csv  exists


## DATAFRAME DEL FITXER smiles.csv

Creem un dataframe del fitxer smiles.csv obtingut de la base de dades Sabio-RK i el llegim. El document separa la informació per tabilació ('\t')

In [153]:
df = pd.read_csv(datafile, sep='\t')
print(df)

       SabioReactionID                              Name       Role  \
0                    1       alpha-D-Glucose 6-phosphate  Substrate   
1                    1       beta-D-Fructose 6-phosphate    Product   
2                    2                               ATP  Substrate   
3                    2  beta-D-Fructose 1,6-bisphosphate    Product   
4                    2                               ADP    Product   
...                ...                               ...        ...   
62291            16513                                H+    Product   
62292            16513                                O2  Substrate   
62293            16513                                CO    Product   
62294            16513      2-Heptyl-3-hydroxy-quinolone  Substrate   
62295            16513        n-Octanoylanthranilic acid    Product   

                                                  Smiles  
0      C([C@@H]1[C@H]([C@@H]([C@H]([C@H](O1)O)O)O)O)O...  
1      C([C@@H]1[C@H]([C@@H](

## Generem un DF curt per a fer proves

Crear smiles reacció:

In [148]:
input='data/smilestest.csv'
output='data/ReactionSmilesTest.csv'

df1 = pd.read_csv(input, sep='\t')

#Volem eliminar tota la informació de les reaccions que tinguin algun valor nul en els seus subtrats i productes a la columna smiles.

#Primer afegim 0 on tronem valors NaN a la columna Smiles del dataframe.
df1['Smiles'] = df1['Smiles'].fillna(0)

# generem una llista buida on guardarem el SabioReactionID dels substrats i productes que s'hagin d'elimianr 
list1 = [] 
#smiles=row["Smiles"] 

# iterem el df1   
for i in range(len(df1)): 

    # Si Smiles es igual a = afegim el SabioReactionID a la list1 
    if df1.loc[i,'Smiles'] == 0: 
        list1.append(df1.loc[i,'SabioReactionID']) 

#Eliminem del df1 tots els substrats i productes que tinguin el SabioReactionID igual que el que s'ha guardat a la llista
df = df1.drop(df1[df1['SabioReactionID'].isin(list1)].index) 
print(df)

#A partir del dataframe sense missing values generem el dataframe curt per fer proves.
#Generem un dataframe buit amb les columnes SabioID i Reaction de 11 files.
df = df.reset_index()  #la funció rest_index() restableix l'índex del dataframe, estableix una llista del 0 fins a la longitud del dataframe
numrows=11
df2 = pd.DataFrame(columns=["SabioID", "Reaction"]) #, index=range(numrows))
#print(df2)

#Creem dues llistes buides amb tants strings com files (en aquest cas 11 numrows=11). Generem una llista R per guardar els reactius i una llista P per guardar els productes.
R=[""]*(numrows+1)
P=[""]*(numrows+1)
isFirstR=[True]*(numrows+1)
isFirstP=[True]*(numrows+1)

def check_smiles(smiles):
    print(smiles)
    if not smiles:
        print('Wrong SMILES for:',smiles)
        return 0
    return 1

#Fem una iteració per cada fila del dataframe amb el command: for index, row in df.iterrows():
for index, row in df.iterrows():
    rid=row['SabioReactionID']
    #print('RID:',rid,isFirstR[rid],isFirstP[rid])
    #irid=int(rid)
    #print(rid)
    #print(type(rid))
    
    name=row["Name"]
    smiles=row["Smiles"]
    if check_smiles(smiles) == 0:
        break
   
    role=row["Role"]
    if role == "Substrate": 
        
        if isFirstR[rid]:
            R[rid]=R[rid] + smiles 
            isFirstR[rid]= False
        else:
            R[rid]=R[rid] + "." + smiles
            
        
    elif role == "Product":
        if isFirstP[rid]:
            P[rid]=P[rid]+smiles
            isFirstP[rid]=False
        else:
            P[rid]=P[rid] +"."+ smiles
    else:
        print("error, name")
    print(R[rid],P[rid])
    df2.at[rid, "SabioID"]=rid
    
for i in range(numrows):
    print(R[i]+">"+P[i])
    df2.at[i, "Reaction"]=R[i]+">"+P[i]

  

df2.drop(df2.tail(2).index,inplace=True)
print(df2)

#obtenim ReactionSmilesTest.csv.
df2.to_csv(output)


    SabioReactionID                              Name       Role  \
0                 1       alpha-D-Glucose 6-phosphate  Substrate   
1                 1       beta-D-Fructose 6-phosphate    Product   
2                 2                               ATP  Substrate   
3                 2  beta-D-Fructose 1,6-bisphosphate    Product   
4                 2                               ADP    Product   
5                 2       beta-D-Fructose 6-phosphate  Substrate   
6                 3      D-Glyceraldehyde 3-phosphate    Product   
7                 3               Glycerone phosphate    Product   
8                 3  beta-D-Fructose 1,6-bisphosphate  Substrate   
9                 4      D-Glyceraldehyde 3-phosphate  Substrate   
10                4               Glycerone phosphate    Product   
11                5                                H+    Product   
12                5                              NADH    Product   
13                5      D-Glyceraldehyde 3-phos

Genrem csv amb els SMILES de cada substrat i producte per separat per poder treballar amb RDKit i obtenir els features. 

In [125]:
input='data/smilestest.csv'
output='data/moleculesSmilesTest.csv'

df1 = pd.read_csv(input, sep='\t')

df1['Smiles'] = df1['Smiles'].fillna(0)
# create an empty list  
list1 = [] 
#smiles=row["Smiles"] 
# iterate over each row of dataframe   
for i in range(len(df1)): 

    # check if Smiles is 0 then append the SabioreactionID to list1  
    if df1.loc[i,'Smiles'] == 0: 
        list1.append(df1.loc[i,'SabioReactionID']) 

df = df1.drop(df1[df1['SabioReactionID'].isin(list1)].index) 
# Print the updated dataframe 
print(df)


dfmolecules = df.drop_duplicates()
print(dfmolecules)
dfmolecules.to_csv(output)

    SabioReactionID                              Name       Role  \
0                 1       alpha-D-Glucose 6-phosphate  Substrate   
1                 1       beta-D-Fructose 6-phosphate    Product   
2                 2                               ATP  Substrate   
3                 2  beta-D-Fructose 1,6-bisphosphate    Product   
4                 2                               ADP    Product   
5                 2       beta-D-Fructose 6-phosphate  Substrate   
6                 3      D-Glyceraldehyde 3-phosphate    Product   
7                 3               Glycerone phosphate    Product   
8                 3  beta-D-Fructose 1,6-bisphosphate  Substrate   
9                 4      D-Glyceraldehyde 3-phosphate  Substrate   
10                4               Glycerone phosphate    Product   
11                5                                H+    Product   
12                5                              NADH    Product   
13                5      D-Glyceraldehyde 3-phos

# RDKit
 És una llibreria de codi obert de química computacional de python. Està dissenyat per a la manipulació de molècules i estructures químiques. RDKit té moltes funcions algunes d'elles són la conversió entre diferents formats de fitxers de molècules, la generació d'estructures moleculars, el càlcul de propietats físico-químiques de les molècules, com ara la solubilitat, el pes molecular, el nombre d'àtoms rotatius, la càrrega, etc. També permet generar visualitzacions 2D i 3D de molècules. Nosaltres l'utilizarem per a obtenir les _features_ , propietats fisico-químiques, que ens serviran de _finguerprints_ per a la generació del model ML. 

 Descriptors RDKit:

- Pes molecular: El pes molecular d'una molècula pot estar relacionat amb la seva activitat enzimàtica, ja que les molècules més grans o més complexes poden tenir més probabilitats d'interaccionar amb enzims específics. 

- Lipofílicitat: Capacitat d'un compost a dissoldre's en lípids o dissolvents no polars.

- Àrea de superfície polar topològica (TPSA) Chem.Descriptors.TPSA(): és una mesura de l'àrea superficial d'una molècula que és polar o capaç de participar en l'enllaç d'hidrogen. La TPSA pot estar relacionada amb l'activitat enzimàtica, ja que molts enzims tenen llocs actius polars o carregats. 

- Nombre d'heteroàtoms. NumHeteroatoms()

- Nombre d'anells. RingCount()

- Nombre d'anells aromàtis. NumAromaticRings()

- Nombre d'àtoms pesants: Chem.Descriptors.HeavyAtomCount(mol)

- Nombre d'anells alifàtics: Chem.Descriptors.NumAliphaticRings(mol)

- Fraction of carbons that are sp3 hybridized: Chem.Descriptors.FractionCSP3(mol)

- Number of chiral centers: Chem.Descriptors.NumChiralCenters(mol)

- Balaban J index: Chem.Descriptors.BalabanJ(mol) : L'índex J de Balaban mesura el grau d'insaturació d'una molècula. Està relacionat amb la reactivitat química i l'estabilitat d'una molècula, així com amb la seva aromaticitat. Les molècules amb índex més alts tendeixen a tenir graus més alts d'insaturació i, per tant, són més reactives mentre que les molècules amb índex més baixos solen ser més estables.

- Number of rotatable bonds: Chem.Descriptors.NumRotatableBonds(mol)

<class 'rdkit.Chem.rdchem.Mol'>


[18:47:23] WARNING: not removing hydrogen atom without neighbors


In [51]:
#df['mol'] = df['mol'].apply(lambda x: Chem.AddHs(x))
#df['num_of_atoms'] = df['mol'].apply(lambda x: x.GetNumAtoms())
#df['num_of_heavy_atoms'] = df['mol'].apply(lambda x: x.GetNumHeavyAtoms())

In [208]:
from rdkit import Chem
from rdkit.Chem import Crippen, Descriptors

df1 = pd.read_csv('data/moleculesSmilesTest.csv', sep=',')
df1.head()
df1['mol']=df1['Smiles'].apply(lambda x: Chem.MolFromSmiles(x)) #transformem la columna "Smiles" en mol per començar a treballar amb RDKit
print(type(df1['mol'][0])) #comprovem que hem genereat la classe rdkit.Chem.rdchem.Mol
from rdkit import Chem
from rdkit.Chem import Crippen, Descriptors
output='data/fingerprintTest.csv'

mols = df1['mol']

def smiles_to_mol(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol


def features(mol,data):
    #Extract properties using RDKit functions
    smiles = Chem.MolToSmiles(mol)
    formula = Chem.rdMolDescriptors.CalcMolFormula(mol)
    mw= Chem.rdMolDescriptors.CalcExactMolWt(mol) #pes molecular
    logp=Chem.Descriptors.MolLogP(mol)
    tpsa = Chem.rdMolDescriptors.CalcTPSA(mol)
    num_rc=Chem.Descriptors.RingCount(mol)
    num_AR=Chem.Descriptors.NumAromaticRings(mol)
    num_HAC=Chem.Descriptors.HeavyAtomCount(mol)
    num_AR=Chem.Descriptors.NumAliphaticRings(mol)
    CSP3=Chem.Descriptors.FractionCSP3(mol)
    BJI=Chem.Descriptors.BalabanJ(mol)
    num_RB=Chem.Descriptors.NumRotatableBonds(mol)
    num_hbd = Chem.rdMolDescriptors.CalcNumHBD(mol)
    num_hba = Chem.rdMolDescriptors.CalcNumHBA(mol)
   

    #Append results in a data list
    data.append([smiles,formula,mw,logp, tpsa, num_rc, num_AR, num_HAC, num_AR, CSP3, BJI, num_RB, num_hbd, num_hba])
    return

data = []

for mol in mols:
    features(mol,data)

#We have a list with information. Creation of a Dataframe:
df= pd.DataFrame(data,columns=['SMILES','Formula','MW','logP',"TPSA", "Rings Count", "Aromatic Rings", "Heavy atoms cout", "Alifatic Rings",  "CSP3",  "BalabanJ", "RotatableBonds", "H-bond donor", "H-bond acceptor"])
df["SabioReactionID"]= df1["SabioReactionID"]
df["Role"]=df1["Role"]
print(df)

df.to_csv(output)



<class 'rdkit.Chem.rdchem.Mol'>
                                               SMILES         Formula  \
0   O=P(O)(O)OC[C@H]1O[C@H](O)[C@H](O)[C@@H](O)[C@...        C6H13O9P   
1      O=P(O)(O)OC[C@H]1O[C@](O)(CO)[C@@H](O)[C@@H]1O        C6H13O9P   
2   Nc1ncnc2c1ncn2C1OC(COP(=O)(O)OP(=O)(O)OP(=O)(O...   C10H16N5O13P3   
3              O=P(O)(O)OCC1OC(O)(COP(=O)(O)O)C(O)C1O      C6H14O12P2   
4     Nc1ncnc2c1ncn2C1OC(COP(=O)(O)OP(=O)(O)O)C(O)C1O   C10H15N5O10P2   
5      O=P(O)(O)OC[C@H]1O[C@](O)(CO)[C@@H](O)[C@@H]1O        C6H13O9P   
6                                  O=CC(O)COP(=O)(O)O         C3H7O6P   
7                                  O=C(CO)COP(=O)(O)O         C3H7O6P   
8              O=P(O)(O)OCC1OC(O)(COP(=O)(O)O)C(O)C1O      C6H14O12P2   
9                                  O=CC(O)COP(=O)(O)O         C3H7O6P   
10                                 O=C(CO)COP(=O)(O)O         C3H7O6P   
11                                                [H]               H   
12  NC(=O)C1=CN(C2O

[11:33:18] WARNING: not removing hydrogen atom without neighbors


Generem un fitxer csv per introduir al programa Chemprop. El fitxer ha de tenir la columna: ReactionSmiles Sabio ID i la suma de fingerprints dels seus substrats i productes. 

In [176]:
input='data/FingerprintTest.csv'
output='data/ChempropTest.csv'

# Sumem els valors de MW
df1 = pd.read_csv(input, sep=',')
SMW = {}
for index, row in df1.iterrows():
    rid=row['SabioReactionID']
    role=row["Role"]
    MW=row["MW"]
    if rid not in SMW:
        SMW[rid] = 0
    if role == "Substrate":
        SMW[rid] += MW
    
    
print(SMW)



{1: 260.02971862600003, 2: 767.0254640879998, 3: 339.9960491479999, 4: 169.998024574, 5: 932.0913180360901, 6: 692.9886846559998, 8: 185.992939194, 9: 595.0117894499998, 10: 687.0591335659998}


In [182]:
# Sumem els valors de lipofilicitat dels substrats:
df1 = pd.read_csv(input, sep=',')


slogP = {}
for index, row in df1.iterrows():
    rid = row['SabioReactionID']
    role = row['Role']
    logP = row['logP']
    if rid not in slogP:
        slogP[rid] = 0
    if role == 'Substrate':
        slogP[rid] += logP

print(slogP)

{1: -3.104399999999999, 2: -4.731800000000001, 3: -2.9858, 4: -1.3445000000000005, 5: -5.288999999999998, 6: -3.087800000000001, 8: -1.4588000000000003, 9: -1.9690000000000007, 10: -4.850400000000001}


In [183]:
# Sumem els valors de lipofilicitat dels productes:
df1 = pd.read_csv(input, sep=',')


plogP = {}
for index, row in df1.iterrows():
    rid = row['SabioReactionID']
    role = row['Role']
    logP = row['logP']
    if rid not in plogP:
        plogP[rid] = 0
    if role == 'Product':
        plogP[rid] += logP

print(plogP)

{1: -3.1028, 2: -4.731800000000001, 3: -2.6874000000000007, 4: -1.3429000000000002, 5: -4.481399999999998, 6: -3.653500000000001, 8: -1.1307, 9: -2.052000000000001, 10: -4.8504000000000005}


In [187]:
# Sumem els valors de TPSA dels substrats i productes (mateix valor):
df1 = pd.read_csv(input, sep=',')


sTPSA = {}
for index, row in df1.iterrows():
    rid = row['SabioReactionID']
    role = row['Role']
    TPSA = row['TPSA']
    if rid not in sTPSA:
        sTPSA[rid] = 0
    if role == 'Substrate':
        sTPSA[rid] += TPSA

print(sTPSA)

{1: 156.91, 2: 436.03999999999996, 3: 203.44, 4: 104.06, 5: 500.0799999999999, 6: 403.42, 8: 124.29000000000002, 9: 333.5, 10: 389.51}


In [188]:
# Sumem els valors de "Rings count" dels substrats:
df1 = pd.read_csv(input, sep=',')


Snum_rc = {}
for index, row in df1.iterrows():
    rid = row['SabioReactionID']
    role = row['Role']
    num_rc = row['Rings Count']
    if rid not in Snum_rc:
        Snum_rc[rid] = 0
    if role == 'Substrate':
        Snum_rc[rid] += num_rc

print(Snum_rc)

{1: 1, 2: 4, 3: 1, 4: 0, 5: 5, 6: 3, 8: 0, 9: 3, 10: 4}


In [189]:
# Sumem els valors de "Rings Count" dels productes:
df1 = pd.read_csv(input, sep=',')


Pnum_rc = {}
for index, row in df1.iterrows():
    rid = row['SabioReactionID']
    role = row['Role']
    num_rc = row['Rings Count']
    if rid not in Pnum_rc:
        Pnum_rc[rid] = 0
    if role == 'Product':
        Pnum_rc[rid] += num_rc

print(Pnum_rc)

{1: 1, 2: 4, 3: 0, 4: 0, 5: 5, 6: 3, 8: 0, 9: 3, 10: 4}


In [193]:
# Sumem els valors de "Aromatic Rings" dels substrats:
df1 = pd.read_csv(input, sep=',')


Snum_ar = {}
for index, row in df1.iterrows():
    rid = row['SabioReactionID']
    role = row['Role']
    num_ar = row['Aromatic Rings']
    if rid not in Snum_ar:
        Snum_ar[rid] = 0
    if role == 'Substrate':
        Snum_ar[rid] += num_ar

print(Snum_ar)

{1: 1, 2: 2, 3: 1, 4: 0, 5: 2, 6: 1, 8: 0, 9: 1, 10: 2}


In [192]:
# Sumem els valors de "Aromatic Rings" dels productes:
df1 = pd.read_csv(input, sep=',')


Pnum_ar = {}
for index, row in df1.iterrows():
    rid = row['SabioReactionID']
    role = row['Role']
    num_ar = row['Aromatic Rings']
    if rid not in Pnum_ar:
        Pnum_ar[rid] = 0
    if role == 'Product':
        Pnum_ar[rid] += num_ar

print(Pnum_ar)

{1: 1, 2: 2, 3: 0, 4: 0, 5: 3, 6: 1, 8: 0, 9: 1, 10: 2}


In [197]:
# Sumem els valors de "Heavy atoms cout" dels substrast:
df1 = pd.read_csv(input, sep=',')


Snum_HAC = {}
for index, row in df1.iterrows():
    rid = row['SabioReactionID']
    role = row['Role']
    num_HAC = row["Heavy atoms cout"]
    if rid not in Snum_HAC:
        Snum_HAC[rid] = 0
    if role == 'Substrate':
       Snum_HAC[rid] += num_HAC

print(Snum_HAC)


{1: 16, 2: 47, 3: 20, 4: 10, 5: 59, 6: 42, 8: 11, 9: 37, 10: 43}


In [198]:
# Sumem els valors de "Heavy atoms cout" dels productes:
df1 = pd.read_csv(input, sep=',')


Pnum_HAC = {}
for index, row in df1.iterrows():
    rid = row['SabioReactionID']
    role = row['Role']
    num_HAC = row["Heavy atoms cout"]
    if rid not in Pnum_HAC:
        Pnum_HAC[rid] = 0
    if role == 'Product':
       Pnum_HAC[rid] += num_HAC

print(Pnum_HAC)

{1: 16, 2: 47, 3: 20, 4: 10, 5: 59, 6: 42, 8: 11, 9: 37, 10: 43}


In [199]:
# Sumem els valors de "Alifatic Rings" dels substrats:
df1 = pd.read_csv(input, sep=',')


Snum_AR = {}
for index, row in df1.iterrows():
    rid = row['SabioReactionID']
    role = row['Role']
    num_AR = row["Alifatic Rings"]
    if rid not in Snum_AR:
        Snum_AR[rid] = 0
    if role == 'Substrate':
       Snum_AR[rid] += num_AR

print(Snum_AR)

{1: 1, 2: 2, 3: 1, 4: 0, 5: 2, 6: 1, 8: 0, 9: 1, 10: 2}


In [200]:
# Sumem els valors de "Alifatic Rings" dels productes:
df1 = pd.read_csv(input, sep=',')


Pnum_AR = {}
for index, row in df1.iterrows():
    rid = row['SabioReactionID']
    role = row['Role']
    num_AR = row["Alifatic Rings"]
    if rid not in Pnum_AR:
        Pnum_AR[rid] = 0
    if role == 'Product':
       Pnum_AR[rid] += num_AR

print(Pnum_AR)

{1: 1, 2: 2, 3: 0, 4: 0, 5: 3, 6: 1, 8: 0, 9: 1, 10: 2}


In [201]:
# Sumem els valors de "CSP3" dels substrats:
df1 = pd.read_csv(input, sep=',')

sCSP3 = {}
for index, row in df1.iterrows():
    rid = row['SabioReactionID']
    role = row['Role']
    CSP3 = row["CSP3"]
    if rid not in sCSP3:
        sCSP3[rid] = 0
    if role == 'Substrate':
       sCSP3[rid] += CSP3

print(sCSP3)

{1: 1.0, 2: 1.5, 3: 1.0, 4: 0.6666666666666666, 5: 1.1428571428571428, 6: 1.1666666666666665, 8: 0.6666666666666666, 9: 0.8333333333333333, 10: 1.5}


In [202]:
# Sumem els valors de "CSP3" dels productes:
df1 = pd.read_csv(input, sep=',')

pCSP3 = {}
for index, row in df1.iterrows():
    rid = row['SabioReactionID']
    role = row['Role']
    CSP3 = row["CSP3"]
    if rid not in pCSP3:
        pCSP3[rid] = 0
    if role == 'Product':
       pCSP3[rid] += CSP3

print(pCSP3)

{1: 1.0, 2: 1.5, 3: 1.3333333333333333, 4: 0.6666666666666666, 5: 1.1904761904761905, 6: 1.1666666666666665, 8: 0.0, 9: 0.5, 10: 1.5}


In [203]:
# Sumem els valors de "BalabanJ" dels substrats:
df1 = pd.read_csv(input, sep=',')

sBJ = {}
for index, row in df1.iterrows():
    rid = row['SabioReactionID']
    role = row['Role']
    BJ = row["BalabanJ"]
    if rid not in sBJ :
        sBJ [rid] = 0
    if role == 'Substrate':
       sBJ [rid] += BJ

print(sBJ )

{1: 2.572326379287313, 2: 4.30261058219916, 3: 2.6740286330746827, 4: 3.550748721097319, 5: 8.180443307420967, 6: 5.4103939640235685, 8: 4.101477779413871, 9: 5.257112855243928, 10: 4.323818387262321}


In [205]:
# Sumem els valors de "BalabanJ" dels substrats:
df1 = pd.read_csv(input, sep=',')

pBJ = {}
for index, row in df1.iterrows():
    rid = row['SabioReactionID']
    role = row['Role']
    BJ = row["BalabanJ"]
    if rid not in pBJ :
        pBJ [rid] = 0
    if role == 'Product':
       pBJ [rid] += BJ

print(pBJ )

{1: 2.613201181998435, 2: 4.422680439879217, 3: 7.111275753771958, 4: 3.560527032674639, 5: 5.270757946784707, 6: 5.872214964434617, 8: 4.067302180177984, 9: 5.815953986982519, 10: 4.320978186091848}


In [206]:
# Sumem els valors de "RotatableBonds"  dels substrats:
df1 = pd.read_csv(input, sep=',')

Snum_rb = {}
for index, row in df1.iterrows():
    rid = row['SabioReactionID']
    role = row['Role']
    num_rb = row["RotatableBonds"]
    if rid not in Snum_rb  :
        Snum_rb  [rid] = 0
    if role == 'Substrate':
       Snum_rb [rid] += num_rb

print(Snum_rb)

{1: 3, 2: 12, 3: 6, 4: 4, 5: 15, 6: 12, 8: 4, 9: 9, 10: 9}


In [207]:
# Sumem els valors de "RotatableBonds" dels productes:
df1 = pd.read_csv(input, sep=',')

Pnum_rb = {}
for index, row in df1.iterrows():
    rid = row['SabioReactionID']
    role = row['Role']
    num_rb = row["RotatableBonds"]
    if rid not in Pnum_rb  :
        Pnum_rb  [rid] = 0
    if role == 'Product':
       Pnum_rb [rid] += num_rb

print(Pnum_rb)

{1: 4, 2: 12, 3: 8, 4: 4, 5: 16, 6: 11, 8: 3, 9: 9, 10: 9}


In [209]:
# Sumem els valors de "H-bond donor" dels substrats:
df1 = pd.read_csv(input, sep=',')

Snum_HBD = {}
for index, row in df1.iterrows():
    rid = row['SabioReactionID']
    role = row['Role']
    num_HBD = row["H-bond donor"]
    if rid not in Snum_HBD  :
        Snum_HBD [rid] = 0
    if role == 'Substrate':
       Snum_HBD [rid] += num_HBD

print(Snum_HBD)

{1: 6, 2: 13, 3: 7, 4: 3, 5: 14, 6: 11, 8: 4, 9: 8, 10: 12}


In [210]:
# Sumem els valors de "H-bond donor" dels productes:
df1 = pd.read_csv(input, sep=',')

Pnum_HBD = {}
for index, row in df1.iterrows():
    rid = row['SabioReactionID']
    role = row['Role']
    num_HBD = row["H-bond donor"]
    if rid not in Pnum_HBD  :
        Pnum_HBD [rid] = 0
    if role == 'Product':
       Pnum_HBD [rid] += num_HBD

print(Pnum_HBD)

{1: 6, 2: 13, 3: 6, 4: 3, 5: 13, 6: 11, 8: 3, 9: 9, 10: 12}


In [212]:
# Sumem els valors de "H-bond acceptor" dels substrats:
df1 = pd.read_csv(input, sep=',')

Snum_HBA = {}
for index, row in df1.iterrows():
    rid = row['SabioReactionID']
    role = row['Role']
    num_HBA = row["H-bond acceptor"]
    if rid not in Snum_HBA  :
        Snum_HBA [rid] = 0
    if role == 'Substrate':
       Snum_HBA [rid] += num_HBA

print(Snum_HBA)

{1: 7, 2: 21, 3: 8, 4: 4, 5: 22, 6: 18, 8: 4, 9: 16, 10: 20}


In [211]:
# Sumem els valors de "H-bond acceptor" dels productes:
df1 = pd.read_csv(input, sep=',')

Pnum_HBA = {}
for index, row in df1.iterrows():
    rid = row['SabioReactionID']
    role = row['Role']
    num_HBA = row["H-bond acceptor"]
    if rid not in Pnum_HBA  :
        Pnum_HBA [rid] = 0
    if role == 'Product':
       Pnum_HBA [rid] += num_HBA

print(Pnum_HBA)

{1: 7, 2: 20, 3: 8, 4: 4, 5: 24, 6: 18, 8: 3, 9: 15, 10: 19}
